In [29]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

walker_60_suwon = pd.read_csv("D:\\jupyter_proj\\walker_60_suwon.csv")

In [54]:
walker_60_suwon['도로심각도지수basic']

0       중
1       하
2       중
3       하
4       하
       ..
1449    하
1450    하
1451    하
1452    중
1453    하
Name: 도로심각도지수basic, Length: 1454, dtype: object

In [32]:
Q1 = walker_60_suwon['도로심각도지수basic'].quantile(.25)
Q3 = walker_60_suwon['도로심각도지수basic'].quantile(.75)
Q2 = walker_60_suwon['도로심각도지수basic'].quantile(.5)
Q4 = walker_60_suwon['도로심각도지수basic'].quantile(1)

print("Q1 =",Q1)
print("Q2 =",Q2)
print("Q3 =",Q3)
print("Q4 =",Q4)

Q1 = 0.0068
Q2 = 0.1168
Q3 = 0.1168
Q4 = 1.0068


In [48]:
for i in range(len(walker_60_suwon)):
    if walker_60_suwon['도로심각도지수basic'][i] <= 0.0068:
        walker_60_suwon['도로심각도지수basic'][i] = '하'
    elif walker_60_suwon['도로심각도지수basic'][i] <= 0.1168:
        walker_60_suwon['도로심각도지수basic'][i] = '중'
    else:
        walker_60_suwon['도로심각도지수basic'][i] = '상'

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:

In [50]:
xval = walker_60_suwon[walker_60_suwon.columns.difference(['도로심각도지수basic'])]
yval = walker_60_suwon['도로심각도지수basic']

x_train, x_test, y_train, y_test = train_test_split(xval, yval, train_size=0.7, random_state=97)
pipeline = Pipeline([('clf', RandomForestClassifier(criterion='gini', class_weight = {0:0.3, 1:0.7}))])

In [51]:
parameters = {
    'clf__n_estimators':(100, 2000, 3000),
    'clf__max_depth':(5, 15, 30),
    'clf__min_samples_split':(2,3),
    'clf__min_samples_leaf':(1,2)
}

In [52]:
grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, cv=5, verbose=1, scoring='accuracy')
grid_search.fit(x_train, y_train)
    

print('Best Training score : %0.3f' %grid_search.best_score_)
print('Best parameter set:')
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print('\t$s: %r' %(param_name, best_parameters[param_name]))

Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:    1.0s finished


ValueError: could not convert string to float: '기타불명'